## Download TCGA-LUAD and TCGA-LUSC sliders
*gdc_manifest.2024-04-26.txt*  is the manifest file for sliders, please download via *GDC Data Transfer Tool*.
Besides,*LUAD* , *LUSC* and *Solid_normal_tissue* slides can be splited by *'gdc_sample_sheet.2024-04-28.tsv'* .

In [ ]:
import os
import glob
import shutil

download_dir = r'Z:\研发部\算法组\cbj\MRI\3cls'#lung
dst_dir = r'Z:\研发部\算法组\cbj\MRI\3cls'
cls_list_file = 'gdc_sample_sheet.2024-04-28.tsv'
metadata = open(cls_list_file,'r').read().split('\n')[1:-1]
for line in metadata:
    filename = line.split('\t')[1]
    project = line.split('\t')[4]
    cls = line.split('\t')[-1]
    if 'Tumor' in cls:
        cls=project
    try:
        svspath = glob.glob(os.path.join(download_dir,'*',filename))[0]
        cls = cls.replace(' ','')
        os.makedirs(os.path.join(dst_dir,cls),exist_ok=True)
        shutil.move(svspath,os.path.join(dst_dir,cls,filename))
    except:
        pass

In [5]:
!pip install histolab==0.3.0
!pip install large-image

   ---------------------------------------- 0.0/32.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/32.5 MB 653.6 kB/s eta 0:00:50
   ---------------------------------------- 0.1/32.5 MB 1.1 MB/s eta 0:00:31
   ---------------------------------------- 0.1/32.5 MB 722.1 kB/s eta 0:00:45
   ---------------------------------------- 0.1/32.5 MB 722.1 kB/s eta 0:00:45
   ---------------------------------------- 0.1/32.5 MB 532.5 kB/s eta 0:01:01
   ---------------------------------------- 0.2/32.5 MB 482.7 kB/s eta 0:01:08
   ---------------------------------------- 0.2/32.5 MB 551.6 kB/s eta 0:00:59
   ---------------------------------------- 0.2/32.5 MB 551.6 kB/s eta 0:00:59
   ---------------------------------------- 0.2/32.5 MB 551.6 kB/s eta 0:00:59
   ---------------------------------------- 0.2/32.5 MB 393.0 kB/s eta 0:01:23
   ---------------------------------------- 0.2/32.5 MB 366.4 kB/s eta 0:01:29
   ---------------------------------------- 0.2/32.5 MB 366.4 k

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anndata 0.9.2 requires h5py>=3, but you have h5py 2.10.0 which is incompatible.
numba 0.58.1 requires numpy<1.27,>=1.22, but you have numpy 1.21.4 which is incompatible.
scanpy 1.9.8 requires h5py>=3, but you have h5py 2.10.0 which is incompatible.
tensorflow-gpu 2.4.0 requires numpy~=1.19.2, but you have numpy 1.21.4 which is incompatible.
tensorflow-gpu 2.4.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.10.0 which is incompatible.


   ---------------- ----------------------- 5.8/14.0 MB 92.5 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.5 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.5 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.0 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.0 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.0 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.0 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.0 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 92.0 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 91.3 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 91.3 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 91.3 kB/s eta 0:01:30
   ---------------- ----------------------- 5.8/14.0 MB 91.3 kB/

In [ ]:
import os
import tqdm, glob
import utils.tcga_util as tu
from histolab.slide import Slide
from histolab.tiler import ScoreTiler
from histolab.scorer import NucleiScorer, CellularityScorer
from histolab.masks import TissueMask
import histolab.filters.image_filters as imf
import histolab.filters.morphological_filters as mof
from histolab.filters.image_filters import BlueFilter, BluePenFilter, GreenFilter, GreenPenFilter, RedPenFilter


def histolab_func(input_path,processed_path):

    tcga_slide = Slide(input_path, processed_path=processed_path, )
    # tcga_slide.show()
    scored_tiles_extractor = ScoreTiler(
        scorer=CellularityScorer(),
        tile_size=(256, 256),
        n_tiles=400,
        level=1,
        check_tissue=True,
        tissue_percent=30.0,
        pixel_overlap=0,  # default
        prefix="scored/",  # save tiles in the "scored" subdirectory of slide's processed_path
        suffix=".png"  # default
    )
    extraction_mask = TissueMask()
    # extraction_mask = tu.CustomMask()
    # extraction_mask.custom_filters = [
    #     tu.BluePenFilter(),  # 对蓝色墨迹进行去除
    #     imf.RgbToGrayscale(),
    #     imf.OtsuThreshold(),  # 阈值分割
    #     mof.BinaryErosion(disk_size=1),  # 腐蚀
    #     mof.BinaryDilation(disk_size=1),  # 膨胀
    #     mof.RemoveSmallHoles(area_threshold=500),
    #     mof.RemoveSmallObjects(min_size=1500), ]
    summary_filename = "summary_ovarian_tiles.csv"
    SUMMARY_PATH = os.path.join(processed_path, summary_filename)
    locate_img = scored_tiles_extractor.locate_tiles(
        slide=tcga_slide,
        scale_factor=24,  # default
        alpha=224,  # default
        outline="green",
        extraction_mask=extraction_mask  # default

    )
    # locate_img.show()
    locate_img.save(os.path.join(processed_path, "locate.png"))

    scored_tiles_extractor.extract(tcga_slide, report_path=SUMMARY_PATH,  extraction_mask=extraction_mask)


if __name__ == '__main__':
    svs_dir = r'Z:\tmp\3cls'
    filelist = glob.glob(os.path.join(svs_dir,'*','*.svs'))
    process_dir = os.path.join(svs_dir,'tiles')
    for svs in tqdm.tqdm(filelist):
        process_path = os.path.join(process_dir,os.path.basename(svs).split('.svs')[0])
        os.makedirs(process_path,exist_ok=True)
        svs = svs.replace('\\','/')
        histolab_func(svs,process_path)

  1%|▏         | 29/2126 [14:01<12:55:04, 22.18s/it]